# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Append project path

In [2]:
import sys

PROJECT_PATH_UTIL = "/content/drive/MyDrive/Colab Notebooks/SCVQA/util"
#                    /content/drive/MyDrive/.../util
sys.path.append(PROJECT_PATH_UTIL)

# Install requirements

In [3]:
!pip install av scikit-video

# Import libraries and py script coded

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import os
import numpy as np
from pathlib import Path
import datetime
from timeit import default_timer as timer

import sys

from dataset import FeatureDataset
from model import SCVQA
from engine import Engine

# Setup parameters

In [5]:
DATABASE = "CSCVQ"
CNN_MODULE = "ResNet50"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
# NUM_WORKERS = os.cpu_count()
NUM_WORKERS = 0
NUM_EPOCHS = 100
SEED = 22035001

FEATURE_DIR = Path(f"/content/drive/MyDrive/Colab Notebooks/SCVQA/feature/{DATABASE}/{CNN_MODULE}/")
#                    /content/drive/MyDrive/.../feature/{DATABASE}/{CNN_MODULE}/

print(
    f"database: {DATABASE}, CNN module: {CNN_MODULE}, device: {DEVICE}, batch_size: {BATCH_SIZE}, num_workers: {NUM_WORKERS}, num_epochs: {NUM_EPOCHS}, seed: {SEED}"
)

database: CSCVQ, CNN module: ResNet50, device: cuda, batch_size: 1, num_workers: 0, num_epochs: 100, seed: 22035001


# Data preparation

In [6]:
feature_data_list = list()

if not os.path.exists(FEATURE_DIR):
    print(f"Video feature not exists in {FEATURE_DIR}/")
    sys.exit()
else:
    count = 0
    os.scandir(FEATURE_DIR)
    video_feature_dir_list = [f.path for f in os.scandir(FEATURE_DIR) if f.is_dir()]

    for video_feature_dir in video_feature_dir_list:
        feature_file = f"{video_feature_dir}/feature.npy"
        mos_file = f"{video_feature_dir}/mos.npy"

        feature = np.load(feature_file)
        feature = torch.from_numpy(feature).to(device=DEVICE)
        # [frames, feature] | Tensor | torch.Size([300, 4096])

        mos = np.load(mos_file)
        mos = torch.from_numpy(mos).to(device=DEVICE).squeeze()
        # mos | Tensor | torch.Size([])

        feature_data_list.append((feature, mos))

TRAIN_SPLIT = int(0.8 * len(feature_data_list))
train_data_list = feature_data_list[:TRAIN_SPLIT]
test_data_list = feature_data_list[TRAIN_SPLIT:]

train_dataset = FeatureDataset(dataset=train_data_list)
test_dataset = FeatureDataset(dataset=test_data_list)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=True,
)
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False,
)

print(
    f"Number of training data: {len(train_dataset)} & testing data: {len(test_dataset)}"
)

Number of training data: 132 & testing data: 33


# Training and testing step

In [7]:
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

model = SCVQA().to(device=DEVICE)

loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

engine = Engine(device=DEVICE, epochs=NUM_EPOCHS)

start_time = timer()
model_results = engine.train(
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
)
end_time = timer()
print(
    f"Total training time: {datetime.timedelta(seconds=int(end_time-start_time))} (Hour:Minute:Second)"
)
# print(model_results)
print(
    f"Total number of epochs: {NUM_EPOCHS} - Total number of results: {len(model_results['train_loss'])}"
)

  0%|          | 0/100 [00:00<?, ?it/s]

[Training] Epoch: 1 | MAE loss: 48.0480 | PCC: -0.0438 | SROCC: -0.0710
[Testing]  Epoch: 1 | MAE loss: 44.7575 | PCC: -0.0275 | SROCC: -0.0463
[Training] Epoch: 2 | MAE loss: 44.7571 | PCC: 0.0945 | SROCC: 0.0744
[Testing]  Epoch: 2 | MAE loss: 41.6206 | PCC: -0.0275 | SROCC: -0.0463
[Training] Epoch: 3 | MAE loss: 41.6499 | PCC: -0.0613 | SROCC: -0.0718
[Testing]  Epoch: 3 | MAE loss: 38.5327 | PCC: -0.0275 | SROCC: -0.0463
[Training] Epoch: 4 | MAE loss: 38.5717 | PCC: 0.0341 | SROCC: 0.0424
[Testing]  Epoch: 4 | MAE loss: 35.4621 | PCC: -0.0275 | SROCC: -0.0463
[Training] Epoch: 5 | MAE loss: 35.5059 | PCC: 0.1214 | SROCC: 0.1066
[Testing]  Epoch: 5 | MAE loss: 32.4004 | PCC: -0.0275 | SROCC: -0.0463
[Training] Epoch: 6 | MAE loss: 32.4471 | PCC: 0.2464 | SROCC: 0.2763
[Testing]  Epoch: 6 | MAE loss: 29.3442 | PCC: -0.0275 | SROCC: -0.0463
[Training] Epoch: 7 | MAE loss: 29.4223 | PCC: 0.0255 | SROCC: 0.0364
[Testing]  Epoch: 7 | MAE loss: 26.4502 | PCC: -0.0275 | SROCC: -0.0463
[T